# RFM clustering using Hierarchical Clustering

## Agglomerative

The principle is simple : at start all points are clusters and then using distance we try step by step to group clusters in bigger clusters.

To do that, the algorithm consider two data:
* the intra cluster variance ;
* the inter cluster variance ;

Like in K-Means, we need to define the number of clusters `n_clusters` and the `linkage` property.

The `linkage` property determines which distance to use between observations.

In [18]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics

customers = pd.read_csv("./../../../output/customers.csv")

y = customers["review_score"]

X_train, X_test, y_train, y_test = train_test_split(
    customers, y, test_size=0.2, random_state=42, stratify=y
)
X = X_test[
    [
        "recency",
        "frequency",
        "monetary",
        "scoring_RFM",
        "review_score",
        "payment_installments",
        "payment_sequential",
        "payment_mean_boleto_value",
        "payment_mean_voucher_value",
        "payment_mean_debit_card_value",
        "payment_mean_credit_card_value",
        "nb_products",
        "mean_delivery_delay",
        "mean_freight_value",
    ]
]

X.head()

,recency,frequency,monetary,scoring_RFM,review_score,payment_installments,payment_sequential,payment_mean_boleto_value,payment_mean_voucher_value,payment_mean_debit_card_value,payment_mean_credit_card_value,nb_products,mean_delivery_delay,mean_freight_value
43517,450.0,0.062500,29.50,3,4.0,1.0,1.0,0.0,0.0,0.0,29.500000,1,8.0,12.60
52729,34.0,0.500000,72.07,12,5.0,1.0,1.0,0.0,0.0,0.0,72.070000,1,5.0,13.07
58266,468.0,0.058824,194.91,6,3.0,1.0,1.0,0.0,0.0,0.0,194.910004,1,5.0,15.01
7281,521.0,0.055556,47.95,3,5.0,4.0,1.0,0.0,0.0,0.0,47.950001,1,7.0,16.05
19387,225.0,0.125000,39.84,7,3.0,1.5,1.5,0.0,0.0,0.0,19.920000,1,20.0,11.85


We scale the data:

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scalar_vars = X[X.select_dtypes(exclude=["object", "category"]).columns]

scaled_customers = scaler.fit_transform(scalar_vars)

scaled_customers

array([[ 1.41751156, -0.69331937, -0.59786197, ..., -0.20870784,
        -0.41759305, -0.4913937 ],
       [-1.33051672,  1.69928597, -0.38657031, ..., -0.20870784,
        -0.72801644, -0.45740001],
       [ 1.53641663, -0.7134253 ,  0.22313295, ..., -0.20870784,
        -0.72801644, -0.31708562],
       ...,
       [-0.28679444, -0.25386124, -0.07749979, ..., -0.20870784,
        -0.93496536, -0.35831201],
       [-0.17449521, -0.3515186 , -0.53040945, ..., -0.20870784,
         0.92757498, -0.31057619],
       [-0.84168477,  0.05864231,  0.01869077, ..., -0.20870784,
         1.1345239 , -0.04151973]])

In [27]:
report = []
linkage_methods = ["ward", "single", "average", "complete"]
affinities = ["euclidean", "l1", "l2", "manhattan", "cosine"]
for nb_clusters in range(2, 8):
    for linkage_method in linkage_methods:
        for affinity in affinities:
            if linkage_method == "ward" and affinity != "euclidean":
                continue
            agg_clustering = AgglomerativeClustering(
                n_clusters=nb_clusters,
                linkage=linkage_method,
                affinity=affinity,
                compute_full_tree=False,
            ).fit(scaled_customers)

            report.append(
                {
                    "nb_clusters": nb_clusters,
                    "silhouette_score": metrics.silhouette_score(
                        scaled_customers, agg_clustering.labels_
                    ),
                    "linkage_method": linkage_method,
                    "affinity": affinity,
                }
            )

report = pd.DataFrame(report)
print(report)
for affinity in affinities:
    report_affinity = report[report.affinity == affinity]
    fig = px.line(
        report_affinity,
        title="Silhouette score versus nb of clusters ({0})".format(affinity),
        x="nb_clusters",
        y="silhouette_score",
        color="linkage_method",
    )

    fig.show()

    nb_clusters  silhouette_score linkage_method   affinity
0             2          0.205836           ward  euclidean
1             2          0.953235         single  euclidean
2             2          0.934758         single         l1
3             2          0.953235         single         l2
4             2          0.934758         single  manhattan
..          ...               ...            ...        ...
91            7          0.859974       complete  euclidean
92            7          0.846288       complete         l1
93            7          0.859974       complete         l2
94            7          0.846288       complete  manhattan
95            7          0.021556       complete     cosine

[96 rows x 4 columns]


Let's display the dendrogram to confirm that the optimal number of clusters is **4** for average method and l2 as affinity parameter.


In [21]:
# from scipy.cluster.hierarchy import linkage, dendrogram
# import matplotlib.pyplot as plt
#
# Z = linkage(scaled_customers, method="ward")
#
# fig = plt.figure(figsize=(16, 8))
# dendrogram(Z, orientation="left", truncate_mode="level", p=4)
#
# plt.show()

In [22]:
agg_clustering = AgglomerativeClustering(
    n_clusters=5, linkage="ward", affinity="euclidean", compute_full_tree=False
).fit(scaled_customers)

X["cluster"] = agg_clustering.labels_
X["cluster"] = X["cluster"].astype("category")

print(X["cluster"].value_counts())

1    10358
3     4130
4     1879
2      796
0      465
Name: cluster, dtype: int64


C:\Users\micka\AppData\Local\Temp\ipykernel_3352\960961023.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\micka\AppData\Local\Temp\ipykernel_3352\960961023.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Clustering visualisation

In [23]:
from sklearn.manifold import TSNE
import plotly.express as px

tsne = TSNE(n_components=2, learning_rate="auto", init="random", n_jobs=-1)
projected_customers = tsne.fit_transform(scaled_customers)

fig = px.scatter(
    projected_customers,
    x=0,
    y=1,
    color=X.cluster,
    labels={"color": "cluster"},
    opacity=0.8,
)
fig.show()

In [28]:
# cluster for value equals to -1 is noise
fig2 = px.scatter_3d(X, x="recency", y="monetary", z="review_score", color="cluster")
fig2.show()

### Conclusions

The hierarchical clustering is in favor of 5 clusters to describe the dataset and have the best performance silhouette score but a really poor clustering performance.

We can't use it as the best model.